# DataRelay Sample
### I can document my code using markdown!

In [2]:
ClientId = dbutils.secrets.get(scope = "Lambda", key = "ClientId")
ClientSecret = dbutils.secrets.get(scope = "Lambda", key = "ClientSecret")
TenantName = dbutils.secrets.get(scope = "Lambda", key = "TenantName")

In [3]:
spark.conf.set("dfs.adls.oauth2.access.token.provider.type", "ClientCredential")
spark.conf.set("dfs.adls.oauth2.client.id", ClientId)
spark.conf.set("dfs.adls.oauth2.credential", ClientSecret)
spark.conf.set("dfs.adls.oauth2.refresh.url", "https://login.microsoftonline.com/"+TenantName+"/oauth2/token")

In [4]:
## Load Data Frame ##
taxidf = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load("adl://sawlake.azuredatalakestore.net/RAW/Public/Taxi/NewYork/2018/Sample.csv")

In [5]:
taxiPd = taxidf.toPandas()

taxiagg = taxiPd.groupby(['passenger_count'])['total_amount', 'trip_distance'].sum()

taxiagg

Out[ 4 ]: 
 total_amount trip_distance
passenger_count 
0 72.80 17.90
1 1906.79 339.38
2 1162.26 202.48
3 108.55 20.90
4 132.67 19.83
5 119.41 16.84
6 61.06 5.75

In [6]:
tdf = sqlContext.createDataFrame(taxiagg).write.format("parquet").save("adl://sawlake.azuredatalakestore.net/BASE/Taxis/summary.parquet")